In [0]:
from pyspark.sql.functions import*
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, ArrayType
from pyspark.sql.functions import current_timestamp

In [0]:
def read_data_from_bronze(path,format_name,**schema):
    return spark.read.format(format_name).options(**schema).load(path)

In [0]:
def select_columns(read_data_cpq, columns):
    return read_data_cpq.select(*columns)

In [0]:
customefields_cpq_list = ["quoteNumber", "quoteId", "CustomFields", "load_data"]
involvedparties_cpq_list=["quoteNumber", "quoteId", "InvolvedParties", "load_data"]
amount_columns_list = ["quoteNumber","quoteId","AverageGrossMarginPercent","Cost","TotalAmount","TotalListPrice","TotalNetPrice",
"TotalProductDiscountAmount","load_data"]
cpq_header_columns_list = ["quoteNumber","quoteId","CurrencyCode","DateCreated","DateModified","DistributionChannel","Division",
"EffectiveDate","ErrorMessage","ExternalId","ExternalSystemId","IsPrimary","MarketCode","MarketId","OpportunityId","OpportunityName",
"Origin","OwnerId","PriceBookId","QuoteTables","RevisionNumber","StatINDIAId","StatINDIAName","StatINRId","StatINRName","StatusId",
"StatusName","load_data"]

In [0]:
def explode_column(dataframe, column_name):
    return dataframe.withColumn(column_name, explode(column_name))

In [0]:
def transform_exploded_column(dataframe,customfields,content,name):
    return dataframe.select("*",col(customfields).getItem(content).alias("content"),col(customfields).getItem(name).alias("Name")).drop(customfields)

In [0]:
def split_and_select_columns(dataframe, column_name,splitted_data):
    split_df = dataframe.withColumn("splitted_data", col(column_name))
    return split_df.select("*", "splitted_data.*").drop("splitted_data").drop(column_name)

In [0]:
def pivot_df(df,column_name,column_value):
    for i in range (len(column_name)):
       df= df.withColumn(column_name[i]+"_"+column_value[0],col(column_name[i]).getField(column_value[0]))\
        .withColumn(column_name[i]+"_"+column_value[1],col(column_name[i]).getField(column_value[1])).drop(column_name[i])
    return df

In [0]:
def create_folder(folder_name):
    for i in folder_name:
        dbutils.fs.mkdirs(f"/FileStore/{i}")   

In [0]:
def write_data_silver(df, path):
     df.write.format("delta").mode("overwrite") .option("mergeSchema", "true") .save(path)